**Purpose of the notebook**: to train megnet on defects and
  1. Save weights for the second track
  2. Save predictions for boosting algorithm

In [1]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import tensorflow as tf
from megnet.models import MEGNetModel
from megnet.data.crystal import CrystalGraph
from scripts.utils import read_json_structures, RootPath
from sklearn.model_selection import train_test_split

/Users/tomatoparetogmail.com/Desktop/Github/idao_22/env/lib/python3.8/site-packages/pymatgen/core/__init__.py:49: UserWarning: Error loading .pmgrc.yaml: 'NoneType' object is not iterable. You may need to reconfigure your yaml file.
  warnings.warn(f"Error loading .pmgrc.yaml: {ex}. You may need to reconfigure your yaml file.")


Loading defects of the structures

In [2]:
public_path = RootPath('data/train/defects/pymatgen', must_exist=True)
private_path = RootPath('data/eval/defects/pymatgen', must_exist=True)
targets_path = RootPath('data/dichalcogenides_public/targets.csv', must_exist=True)

df_public = read_json_structures(public_path).merge(
    pd.read_csv(targets_path), on=['_id']).rename({'structure': 'defects'}, axis=1)
df_private = read_json_structures(private_path).rename({'structure': 'defects'}, axis=1)


2966it [00:01, 1685.05it/s]
2967it [00:01, 1773.85it/s]


preparations for training

In [23]:
def energy_within_threshold(prediction, target):
    success = tf.math.count_nonzero(tf.math.abs(target - prediction) < 0.02)
    return success / tf.cast(tf.size(target), tf.int64)


X_train, X_test, y_train, y_test = train_test_split(df_public['defects'],
                                                    df_public['band_gap'],
                                                    test_size=0.2, random_state=666)

### Training process

Note that we choose *r_cutoff* to equal 20, because defects are usually represented by isolated
atoms, which could be anywhere on the structure space. We also select 15 blocks of megnet with 
200 dimensional graph representation. 

In [ ]:
nfeat_bond = 200
r_cutoff = 20  # very important
gaussian_centers = np.linspace(0, r_cutoff + 1, nfeat_bond)
gaussian_width = 0.5
graph_converter = CrystalGraph(cutoff=r_cutoff)

model = MEGNetModel(graph_converter=graph_converter,
                    centers=gaussian_centers,
                    width=gaussian_width,
                    npass=2,
                    nblocks=15,
                    metrics=energy_within_threshold)

model.train(X_train, y_train, validation_structures=X_test, validation_targets=y_test, epochs=1500)

after training we get weights and megnet predictions and save them for further usage:

In [141]:
weights_name = 'megnet_on_defects.hdf5'
_save_to_path = RootPath(f'models/megnet/defects/{weights_name}', must_exist=True)

model.save_model(_save_to_path)
model.load_weights(_save_to_path)

predictions:

In [ ]:
predictions_name = 'megnet_public_predictions.csv'
_save_to_path = RootPath(f'predictions/{predictions_name}')

df_public['predictions'] = model.predict_structures(df_public['defects'])
df_public[['_id', 'predictions']].rename(
    {'_id': 'id'}, axis=1).set_index('id').to_csv(_save_to_path)


In [ ]:
predictions_name = 'megnet_private_predictions.csv'
_save_to_path = RootPath(f'predictions/{predictions_name}')

df_private['predictions'] = model.predict_structures(df_private['defects'])
df_private[['_id', 'predictions']].rename(
    {'_id': 'id'}, axis=1).set_index('id').to_csv(_save_to_path)
